<a href="https://colab.research.google.com/github/SAMYSOSERIOUS/Best-Machine-Learning-Projects-for-Beginners/blob/main/10_Sentiment_Analysis_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup the Environment

In [1]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, SimpleRNN, Activation, Dropout, Conv1D
from tensorflow.keras.layers import Embedding, Flatten, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np
import spacy
from sklearn.metrics import classification_report

In [2]:
# Fix Colab bug: https://github.com/googlecolab/colabtools/issues/3409
import locale
locale.getpreferredencoding = lambda do_setlocale: "UTF-8"

## Exploratory Data Analysis

In [3]:
data = pd.read_csv("https://storage.googleapis.com/adsa-data/sentiment-analysis/tweeter.csv", header=None, encoding='latin-1')
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
# Check for missing values
data.isnull().any()

,0
0,False
1,False
2,False
3,False
4,False
5,False


## Preparing Data

We only care about the tweet text and tweet sentiment information, which stored in the 5th column and 0th column in the dataset. In the sentiment column, 0 represents negative, and 1 represents positive.

We organize the data as data_X contains all the tweet text, data_y contains the labels.

The following code will convert the tweet text data_X to sequence format that will be feed into RNNs

In [5]:
data_X = data[5]
print(data_X)

0        @switchfoot http://twitpic.com/2y1zl - Awww, t...
1        is upset that he can't update his Facebook by ...
2        @Kenichan I dived many times for the ball. Man...
3          my whole body feels itchy and like its on fire 
4        @nationwideclass no, it's not behaving at all....
                               ...                        
19995    Just woke up. Having no school is the best fee...
19996    TheWDB.com - Very cool to hear old Walt interv...
19997    Are you ready for your MoJo Makeover? Ask me f...
19998    Happy 38th Birthday to my boo of alll time!!! ...
19999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: 5, Length: 20000, dtype: object


#### Label:
*   0 -> NEGATIVE
*   2 -> NEUTRAL
*   4 -> POSITIVE

In [6]:
data_y = pd.get_dummies(data[0]).to_numpy()
print(data_y)

[[ True False]
 [ True False]
 [ True False]
 ...
 [False  True]
 [False  True]
 [False  True]]


Splitting Data for Training

In [8]:
# TODO: Split data into train and valid sets
train_set = data.sample(frac=0.8, random_state=0)
valid_set = data.drop(train_set.index)

## Tokenization

In [9]:
MAX_VOCAB = 18000
MAX_LEN = 150
EMBED_SIZE = 200

In [26]:
# TODO: Tokenize inputs
tokenizer = Tokenizer(num_words=MAX_VOCAB)
tokenizer.fit_on_texts(train_set[5])

In [30]:
# Prepare sequences for training and validation
train_sequences = tokenizer.texts_to_sequences(train_set[5])
valid_sequences = tokenizer.texts_to_sequences(valid_set[5])

# Pad sequences to MAX_LEN
train_X = pad_sequences(train_sequences, maxlen=MAX_LEN)
valid_X = pad_sequences(valid_sequences, maxlen=MAX_LEN)

# Prepare labels based on the split data
train_y = data_y[train_set.index]
valid_y = data_y[valid_set.index]

print("Shape of train_X:", train_X.shape)
print("Shape of valid_X:", valid_X.shape)
print("Shape of train_y:", train_y.shape)
print("Shape of valid_y:", valid_y.shape)


Shape of train_X: (16000, 150)
Shape of valid_X: (4000, 150)
Shape of train_y: (16000, 2)
Shape of valid_y: (4000, 2)


In [14]:
train_X

[[119, 7, 66, 382, 1218, 287, 1, 15, 1167, 73, 1070, 13, 6, 788],
 [52, 6, 470, 21, 3, 1274, 10, 606, 1338],
 [66, 172, 49, 87, 98, 5, 66, 184, 460, 9],
 [8072,
  2243,
  2,
  69,
  8073,
  510,
  74,
  142,
  313,
  8074,
  2244,
  81,
  3,
  8075,
  43,
  150,
  61,
  8076],
 [607, 1, 1728, 21, 31, 5, 5283, 11, 146, 519],
 [1, 66, 76, 181, 6, 2245],
 [7, 2860, 8077, 81, 1025, 36, 1418, 49, 5284, 8078],
 [5285, 492, 131, 147, 23, 348, 10, 4, 317, 137, 52, 17, 297, 2861, 2, 38, 2],
 [750, 336, 318, 8079, 11, 4, 238, 95, 4, 371, 2035],
 [20, 48, 55, 1, 730, 2, 82],
 [8080, 731, 8081, 28, 114, 854, 33, 11, 2514, 104],
 [8082, 69, 1219],
 [8083,
  679,
  1168,
  8084,
  39,
  633,
  6,
  940,
  8085,
  4,
  559,
  634,
  129,
  5286,
  48,
  58,
  8,
  2246],
 [7, 121, 192, 91, 119, 2862, 176],
 [420,
  11,
  3,
  119,
  1026,
  2863,
  51,
  350,
  1,
  38,
  97,
  9,
  322,
  85,
  8,
  253,
  2,
  22,
  40,
  45,
  248,
  216,
  12,
  45,
  1169],
 [10,
  436,
  11,
  1729,
  1,
  184,

## Preparing Word Embeddings using the GloVe Model

In [22]:
!pip install gensim

In [24]:
import gensim.downloader as api

# Load the twitter embeddings model. This model is trained on 2 billion tweets, which contains 27 billion tokens, 1.2 million vocabs.
# might take a while
glove_model = api.load("glove-twitter-200")

In [28]:
# calcultaete number of words
nb_words = len(tokenizer.word_index) + 1
print('All words: ', nb_words)

# obtain the word embedding matrix
embedding_matrix = np.zeros((nb_words, EMBED_SIZE))
for word, i in tokenizer.word_index.items():
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

All words:  26022
Null word embeddings: 10326


**Explanation of the steps performed till now**

Tweets: Is upset that he can't update his Facebook..

Expected Input to RNN model -
Is - Embeddings [200] (32)

upset - Embeddings [200] (450)

that - Embeddings [200] (43)

he - Embeddings [200] (56)

1. Vocabulary of all tweets: 30257 unique tokens
2. Unique token IDs: ID (1, 2, 3, 4... for all the 30257 tokens)
3. Tweets represented as the sequence of IDs [32 450 43 56 ...]

Padding:
"Commonly in RNN's, we take the final output or hidden state and use this to make a prediction (or do whatever task we are trying to do).
If we send a bunch of 0's to the RNN before taking the final output (i.e. 'post' padding as you describe), then the hidden state of the network at the final word in the sentence would likely get 'flushed out' to some extent by all the zero inputs that come after this word.
So intuitively, this might be why pre-padding is more popular/effective." - [link](https://stackoverflow.com/questions/46298793/how-does-choosing-between-pre-and-post-zero-padding-of-sequences-impact-results)

Padding for RNNs - [Link](https://datascience.stackexchange.com/questions/49168/padding-sequences-for-neural-sequence-models-rnns)

[Paper](https://arxiv.org/abs/1903.07288)





## Training and Evaluation


Train and evaluate the SimpleRNN, LSTM, and GRU networks on our prepared dataset.

We are using the pre-trained word embeddings from the glove.twitter.27B.200d.txt data. Using the pre-trained word embeddings as weights for the Embedding layer leads to better results and faster convergence.

We set each models to run 20 epochs, but we also set EarlyStopping rules to prevent overfitting. The results of the SimpleRNN, LSTM, GRU models can be seen below.

In [31]:
model_rnn = Sequential()
model_rnn.add(Embedding(nb_words, EMBED_SIZE, weights=[embedding_matrix], input_length=MAX_LEN, trainable = False))

# TODO: Add a SimpleRNN layer
model_rnn.add(SimpleRNN(100))

model_rnn.add(Dense(2, activation='softmax'))
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_rnn.fit(train_X, train_y, epochs=20, batch_size=120,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))

predictions = model_rnn.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.6057 - loss: 0.6634 - val_accuracy: 0.6683 - val_loss: 0.6192
Epoch 2/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.7284 - loss: 0.5386 - val_accuracy: 0.6607 - val_loss: 0.6371
Epoch 3/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.7214 - loss: 0.5482 - val_accuracy: 0.6005 - val_loss: 0.7090
Epoch 4/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.6869 - loss: 0.5882 - val_accuracy: 0.7132 - val_loss: 0.5591
Epoch 5/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7609 - loss: 0.4961 - val_accuracy: 0.7320 - val_loss: 0.5414
Epoch 6/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7702 - loss: 0.4797 - val_accuracy: 0.7218 - val_loss: 0.5489
Epoch 7/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7841 - loss: 0.4640 - val_accuracy: 0.7380 - val_loss: 0.5327
Epoch 8/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7775 - loss: 0.4688 - val_accuracy: 0.727

## LSTM and GRUs

In [33]:
# TODO: Train a LSTM model by replacing the SimpleRNN layer with a LSTM layer
model_lstm = Sequential()
model_lstm.add(Embedding(nb_words, EMBED_SIZE, weights=[embedding_matrix], input_length=MAX_LEN, trainable = False))
model_lstm.add(LSTM(100))

model_lstm.add(Dense(2, activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_lstm.fit(train_X, train_y, epochs=20, batch_size=120,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))

predictions = model_lstm.predict(valid_X)
predictions = predictions.argmax(axis=1)
# TODO: Print a classification report for the model
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.6379 - loss: 0.6252 - val_accuracy: 0.7487 - val_loss: 0.5123
Epoch 2/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7672 - loss: 0.4837 - val_accuracy: 0.7552 - val_loss: 0.5011
Epoch 3/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7820 - loss: 0.4626 - val_accuracy: 0.7570 - val_loss: 0.4886
Epoch 4/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.7966 - loss: 0.4335 - val_accuracy: 0.7648 - val_loss: 0.4800
Epoch 5/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8044 - loss: 0.4177 - val_accuracy: 0.7615 - val_loss: 0.4886
Epoch 6/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8262 - loss: 0.3809 - val_accuracy: 0.7525 - val_loss: 0.5104
Epoch 7/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8347 - loss: 0.3603 - val_accuracy: 0.7670 - val_loss: 0.5018
Epoch 8/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8538 - loss: 0.3318 - val_accuracy: 0.765

In [34]:
# TODO: Train a GRU model by replacing the SimpleRNN layer with a GRU layer
model_gru = Sequential()
model_gru.add(Embedding(nb_words, EMBED_SIZE, weights=[embedding_matrix], input_length=MAX_LEN, trainable = False))
model_gru.add(GRU(100))

model_gru.add(Dense(2, activation='softmax'))
model_gru.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_gru.fit(train_X, train_y, epochs=20, batch_size=120,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))

predictions = model_gru.predict(valid_X)
predictions = predictions.argmax(axis=1)
# TODO: Print a classification report for the model
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


134/134 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - accuracy: 0.6706 - loss: 0.5966 - val_accuracy: 0.7545 - val_loss: 0.5044
Epoch 2/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7720 - loss: 0.4770 - val_accuracy: 0.7600 - val_loss: 0.4908
Epoch 3/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.7820 - loss: 0.4590 - val_accuracy: 0.7585 - val_loss: 0.4845
Epoch 4/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.7905 - loss: 0.4404 - val_accuracy: 0.7722 - val_loss: 0.4799
Epoch 5/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8102 - loss: 0.4161 - val_accuracy: 0.7745 - val_loss: 0.4729
Epoch 6/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.8164 - loss: 0.4072 - val_accuracy: 0.7722 - val_loss: 0.4841
Epoch 7/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8283 - loss: 0.3813 - val_accuracy: 0.7720 - val_loss: 0.4888
Epoch 8/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8441 - loss: 0.3481 - val_accuracy: 0.769

## Evaluation

In [35]:
import time

def predict(model, text):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=MAX_LEN)
    # Predict
    score = model.predict([x_test])[0]

    return {"NEGATIVE": score[0], "POSITIVE": score[1],
       "elapsed_time": time.time()-start_at}

In [37]:
# TODO: Try few sentences to check the models
predict(model_lstm,'Tunisia is winning the world cup')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


{'NEGATIVE': np.float32(0.026470236),
 'POSITIVE': np.float32(0.97352976),
 'elapsed_time': 0.19471955299377441}

## Pre-trained Word Embeddings

Try training the RNNs with word embeddings but without the pre-trained weight and compare the results with the pre-trained model.


In [38]:
# TODO: Remove embedding_matrix and set trainable=TRUE
model_rnn = Sequential()
model_rnn.add(Embedding(nb_words, EMBED_SIZE, input_length=MAX_LEN, trainable = True))

# TODO: Add a SimpleRNN layer
model_rnn.add(SimpleRNN(100))

model_rnn.add(Dense(2, activation='softmax'))
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_rnn.fit(train_X, train_y, epochs=20, batch_size=120,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))

predictions = model_rnn.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


134/134 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.6024 - loss: 0.6519 - val_accuracy: 0.7007 - val_loss: 0.5601
Epoch 2/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.8428 - loss: 0.3684 - val_accuracy: 0.7347 - val_loss: 0.5796
Epoch 3/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9631 - loss: 0.1149 - val_accuracy: 0.7185 - val_loss: 0.7240
Epoch 4/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9887 - loss: 0.0416 - val_accuracy: 0.7285 - val_loss: 0.8569
Epoch 5/20
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9964 - loss: 0.0183 - val_accuracy: 0.7160 - val_loss: 0.9893
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
              precision    recall  f1-score   support

           0       0.73      0.70      0.71      2034
           1       0.70      0.73      0.72      1966

    accuracy                           0.72      4000
   macro avg       0.72      0.72      0.72      4000
weighted avg       0.72      0.72      0.72      4000